In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_curve, auc

import seaborn as sns
%matplotlib inline

In [2]:
## Vamos a la base y se crea (en principio sin oversampling)
df = pd.read_csv('base.csv')
df.drop(['Unnamed: 0','spotifyID'], axis=1, inplace = True)
y = df['hit']
X = df.drop(['hit'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

### SVC sin SMOTE

In [3]:
svc = SVC()
svc.fit(X_train, y_train)

print("Accuracy on training set: {:.2f}".format(svc.score(X_train, y_train)))
print("Accuracy on test set: {:.2f}".format(svc.score(X_test, y_test)))

Accuracy on training set: 1.00
Accuracy on test set: 0.99


In [4]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

svc = SVC()
svc.fit(X_train_scaled, y_train)

print("Accuracy on training set: {:.2f}".format(svc.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.2f}".format(svc.score(X_test_scaled, y_test)))

Accuracy on training set: 0.99
Accuracy on test set: 0.99


In [5]:
svc = SVC(C=1000)
svc.fit(X_train_scaled, y_train)

print("Accuracy on training set: {:.3f}".format(
    svc.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(svc.score(X_test_scaled, y_test)))

Accuracy on training set: 0.986
Accuracy on test set: 0.985


In [6]:
y_pred = svc.predict(X_test)
roc_auc_score(y_test,y_pred)

0.5

In [7]:
confusion = confusion_matrix(y_test, y_pred)
print(classification_report(y_test,y_pred), confusion)

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7026
          1       0.00      0.00      0.00       105

avg / total       0.97      0.99      0.98      7131
 [[7026    0]
 [ 105    0]]


/Users/hernangoldenberg/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### SMOTE

La influencia que posee el desbalanceo es importante. Se repetirá el análisis con Oversampling

In [8]:
sm = SMOTE(random_state=1)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

svc.fit(X_train_res,y_train_res)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
print("Accuracy on training set: {:.3f}".format(
    svc.score(X_train_res, y_train_res)))
print("Accuracy on test set: {:.3f}".format(svc.score(X_test_scaled, y_test)))

Accuracy on training set: 1.000
Accuracy on test set: 0.985


In [10]:
y_pred = svc.predict(X_test)
roc_auc_score(y_test,y_pred)

0.5059594973770891

In [11]:
confusion = confusion_matrix(y_test, y_pred)
print(classification_report(y_test,y_pred), confusion)

             precision    recall  f1-score   support

          0       0.99      0.98      0.98      7026
          1       0.03      0.03      0.03       105

avg / total       0.97      0.97      0.97      7131
 [[6909  117]
 [ 102    3]]


Al no diferenciarse mucho de 0.5 el ROC y tener un precisión, un recall y f1-score del 3%.  El predict proba no puede calcularse, y se estima un modelo **malo** a través del area bajo la curva ROC.

### SVC balanceado y CV

In [12]:
from sklearn.model_selection import cross_val_score
svc_C = [0.5, 1, 1.5, 2, 2.5]
svc_gamma = [0.05, 0.1, 0.5, 0.8, 0.95]
for c in svc_C:
    for g in svc_gamma:
        model_svc = SVC(kernel='rbf', class_weight='balanced', C=c, gamma = g)
        print (np.mean(cross_val_score(model_svc,X_train,y_train,cv=5)),' gamma:',g,' C:',c)

0.985564365364033  gamma: 0.05  C: 0.5
0.9856334022987931  gamma: 0.1  C: 0.5
0.9857024869274632  gamma: 0.5  C: 0.5
0.9857024869274632  gamma: 0.8  C: 0.5
0.9857024869274632  gamma: 0.95  C: 0.5
0.985564365364033  gamma: 0.05  C: 1
0.9856334022987931  gamma: 0.1  C: 1
0.9857024869274632  gamma: 0.5  C: 1
0.9857024869274632  gamma: 0.8  C: 1
0.9857024869274632  gamma: 0.95  C: 1
0.985564365364033  gamma: 0.05  C: 1.5
0.9856334022987931  gamma: 0.1  C: 1.5
0.9857024869274632  gamma: 0.5  C: 1.5
0.9857024869274632  gamma: 0.8  C: 1.5
0.9857024869274632  gamma: 0.95  C: 1.5
0.985564365364033  gamma: 0.05  C: 2
0.9856334022987931  gamma: 0.1  C: 2
0.9857024869274632  gamma: 0.5  C: 2
0.9857024869274632  gamma: 0.8  C: 2
0.9857024869274632  gamma: 0.95  C: 2
0.985564365364033  gamma: 0.05  C: 2.5
0.9856334022987931  gamma: 0.1  C: 2.5
0.9857024869274632  gamma: 0.5  C: 2.5
0.9857024869274632  gamma: 0.8  C: 2.5
0.9857024869274632  gamma: 0.95  C: 2.5


In [13]:
model_svc_optimo = SVC(kernel='rbf', class_weight='balanced', C=0.5, gamma = 0.5)

In [14]:
model_svc_optimo.fit(X_train,y_train)

SVC(C=0.5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
y_pred = model_svc_optimo.predict(X_test)

In [16]:
roc_auc_score(y_test,y_pred)

0.5

Al ser igual a 0.5, ni se calcula el area al tener en claro lo **malo** del modelo.